Alex Kimball (u1195729)  <br>
Faisal Karim (u1418923)

## This project's goal is to observe the world demopgraphy and the impact of literacy rate on many aspects. 

### We sourced our data from OurWorldinData.org and gapminder.org. The loaded file below represent the raw data. The data set named as "Other Data - Copy.csv" is a amalagamation of multiple data sources from gapminder and ourworldindata. As we slowly accumulated this data set it needed to be cleaned by hand and with the help of chatGPT. As the country name in those each datasets were different It was not possible to just attach them with the help of Panda. This is how it was done:
            

####    Step 1: Upload the new dataset in ChatGPT with the previous dataset were I want it merge. We had to use 2011 dataset as that was the latest one available for literacy rate (But for the heatmap we used the latest 2021 data). Prompt: From the newdataset.csv take the 2011 column and attach it olddataset.csv using country name.

#### Step 2: After ChatGPT gave us the new one. We had to do some correction by hand. For example some data set had USA and some had United States. So, a bit of manual clean up was required


###

### The datasets below are our initial raw datasets. The datasets from gapminder had lots of region based data. We had to match it with our GeoJson data too. So after some digging around we were able to remove the the country that we won't be showing in our heat map. The datasets from ourworldindata.com was mainly used for the heatmap of world.

In [15]:
import pandas as pd
import json
import geopandas as gpd
import numpy as np

# Load the CSV file

pop_raw = pd.read_csv('./population-and-demography.csv')
male_pop_raw = pd.read_csv('./Male - population-and-demography.csv')
female_pop_raw = pd.read_csv('./female - population-and-demography.csv')
life_expec_raw = pd.read_csv('./life expectancy - population-and-demography.csv')
death_rate_raw = pd.read_csv('./death rate - population-and-demography.csv')
birth_rate_raw = pd.read_csv('./birth rate - population-and-demography.csv')
child_death_raw = pd.read_csv('./child mortality rate - population-and-demography.csv')
other_data_raw = pd.read_csv('./Other data - Copy.csv')


                   
# List of non-country entries created by chat GPT 
non_countries = [
    'Africa (UN)', 'Asia (UN)', 'Europe (UN)', 'High-income countries', 'Land-locked developing countries (LLDC)', 
    'Latin America and the Caribbean (UN)', 'Least developed countries', 'Less developed regions', 
    'Less developed regions, excluding China', 'Less developed regions, excluding least developed countries', 
    'Low-income countries', 'Lower-middle-income countries', 'More developed regions', 'Northern America (UN)',
    'Oceania (UN)', 'Small island developing states (SIDS)', 'Upper-middle-income countries', 'World', 'Aruba', 'Barbados', 'Burundi', 'Saint Barthelemy', 'Bermuda', 'Brunei',
    'Bonaire Sint Eustatius and Saba', "Cote d'Ivoire", 'Curacao', 'Falkland Islands',
    'French Guiana', 'Guernsey', 'Guam', 'Jersey', 'Kiribati', 'Saint Martin (French part)',
    'Northern Mariana Islands', 'Nauru', 'Niue', 'Reunion', 'Rwanda',
    'Sint Maarten (Dutch part)', 'Tokelau', 'Turkey', 'Kosovo', 'Yemen' 
]

# Filter out the non-country entries
pop = pop_raw[~pop_raw['Country name'].isin(non_countries)]
male_pop = male_pop_raw[~male_pop_raw['Country name'].isin(non_countries)]
female_pop = female_pop_raw[~female_pop_raw['Country name'].isin(non_countries)]
life_expec_raw2 = life_expec_raw[~life_expec_raw['Country name'].isin(non_countries)]
death_rate_raw2 = death_rate_raw[~death_rate_raw['Country name'].isin(non_countries)]
birth_rate_raw2 = birth_rate_raw [~birth_rate_raw ['Country name'].isin(non_countries)]
child_death_raw2 = child_death_raw[~child_death_raw['Country name'].isin(non_countries)]
other_data_raw2 = other_data_raw[~other_data_raw['Country name'].isin(non_countries)]

#Filter out 'Vatican' from some files
death_rate = death_rate_raw2[death_rate_raw2['Country name'] != 'Vatican']
birth_rate = birth_rate_raw2[birth_rate_raw2['Country name'] != 'Vatican']
child_death = child_death_raw2[child_death_raw2['Country name'] != 'Vatican']
life_expec = life_expec_raw2[life_expec_raw2['Country name'] != 'Vatican']
other_data = other_data_raw2[other_data_raw2['Country name'] != 'Vatican']

# other_data.to_csv('./other_data.csv', index=False)
# child_death.to_csv('./child_death.csv', index=False)

### In the next cell We only kept the column that I will use 

In [3]:
pop = pop[['Country name', 'Year', 'Population']]
male_pop = male_pop[['Country name', 'Year', 'Male population']]
female_pop = female_pop[['Country name', 'Year', 'Female population']]

### All the data we had didn't have any ISO3 code with them which we will use to make a link with our worl.json file. So we used chatGPT to create the ISO3 for every countyr. The prompt was "Here is the name of the countires. Now use your knowledge to give me the ISO3 and ISO2 for the countires and give me the list in a CSV file"

In [4]:
iso = pd.read_csv('./Country_with_ISO_Codes.csv')

### Creating a new dataset combining all the datasets from ourworldindata

In [13]:
dataframes = {
    'pop': pop,
    'male_pop': male_pop,
    'female_pop': female_pop,
    'death_rate': death_rate,
    'birth_rate': birth_rate,
    'child_death': child_death
}


columns_to_take = [
    ['Country name', 'Year', 'Population'],
    ['Country name', 'Year', 'Male population'],
    ['Country name', 'Year', 'Female population'],
    ['Country name', 'Year', 'Death rate'],
    ['Country name', 'Year', 'Birth rate'],
    ['Country name', 'Year', 'Child mortality rate']
]

# Initialize an empty DataFrame
final_df = pd.DataFrame()

# Iterate over the datasets and columns
for dataset_name, columns in zip(dataframes.keys(), columns_to_take):
    df = dataframes[dataset_name][columns]
    if final_df.empty:
        final_df = df
    else:
        final_df = final_df.merge(df, on=['Country name', 'Year'], how='inner')


#Merging data with ISO        
final = final_df.merge(iso, on='Country name', how='left')

# final.to_csv('./final.csv', index=False)

### Merging data for heatmap with GEO JSON

In [11]:
# Load the GeoJSON data
world_geo = gpd.read_file('./world.json')

# Filter the population data for the year 2021
final_2021 = final[final['Year'] == 2021]

# Merge the population data with geographical data using ISO3 codes
merged_data = world_geo.merge(final_2021, left_on='id', right_on='ISO3')

# Applying a logarithmic scale to the population data
merged_data['Log_Population'] = np.log1p(merged_data['Population'])

# Convert the merged GeoDataFrame to a GeoJSON format
geojson_data = json.loads(merged_data.to_json())

<class 'geopandas.geodataframe.GeoDataFrame'>


### Later on, for each plot data sets were a bit handeled in the explanatory notebook. In some cases we removed some countries like Australia, Equatorial Guinea, Papua New Guinea to tackle the outliers. 